In [6]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import torch
from transformers import RobertaTokenizer, RobertaModel
from tqdm import tqdm
import torch
import tools as tl
import umap
from sklearn.svm import SVC

In [7]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [8]:
# Load the dataset
file_path = '../../datasets/BBC News/BBC News Train.csv'  # Replace with your actual file path
bbc_df = pd.read_csv(file_path)

# Create the test set (735 rows) and training set (remaining rows)
test_set = bbc_df.sample(n=735, random_state=42).reset_index(drop=True)  # Test set with 735 samples
train_set = bbc_df.drop(test_set.index).reset_index(drop=True)  # Training set with remaining rows

In [10]:
train_set.head(3)

,ArticleId,Text,Category
0,2031,uk young top euro earnings league british chil...,business
1,76,tech helps disabled speed demons an organisati...,tech
2,1860,camera phones are must-haves four times more...,tech


In [11]:
# Initialize distilroberta tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('distilroberta-base')
model = RobertaModel.from_pretrained('distilroberta-base')
# Generate embeddings
print("Generating embeddings...")
embeddings_train = tl.generate_embeddings(train_set['Text'].tolist(), tokenizer, model)
embeddings_test = tl.generate_embeddings(test_set['Text'].tolist(), tokenizer, model)

embeddings_test = torch.tensor(embeddings_test)
embeddings_train = torch.tensor(embeddings_train)
print("Embeddings generated!")

Generating embeddings...


Generating Embeddings: 100%|██████████| 46/46 [00:45<00:00,  1.01batch/s]

Embeddings generated!



/var/folders/87/dhv__9sj0yv1kz0pfv2ds9nh0000gn/T/ipykernel_18189/994341711.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings_test = torch.tensor(embeddings_test)
/var/folders/87/dhv__9sj0yv1kz0pfv2ds9nh0000gn/T/ipykernel_18189/994341711.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings_train = torch.tensor(embeddings_train)


In [12]:
# Fit an SVM model to the reduced embeddings
svm_model = SVC(kernel='linear')
svm_model.fit(embeddings_train, train_set['Category'])

# Predict the labels
predicted_labels = svm_model.predict(embeddings_test)

# Add the predicted labels to the dataframe
test_set['cluster'] = predicted_labels
test_set['embedding'] = embeddings_test.tolist()

train_set['embedding'] = embeddings_train.tolist()

In [14]:
accuracy = (test_set['Category'] == test_set['cluster']).mean()

print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 98.64%
